<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Openai%20SDK/Sessions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The Agents SDK introduces built-in session memory to automatically maintain conversation history across multiple agent runs, eliminating the need to manually handle .to_input_list() between turns.
#This Push comes in support of context managment/mapping as context handling is still kind of difficult in agents framework

In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply() #Make nb capable of running async functions

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [5]:
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled

set_tracing_disabled(True)

client = AsyncOpenAI(api_key=GEMINI_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

model = OpenAIChatCompletionsModel(
    model = "gemini-2.5-flash",
    openai_client = client
)

In [6]:
# Agent Run without Sessions chat history being available.
agent = Agent(name="Assistant",
              instructions = "You are a helpful bot",
              model = model
              )

res = await Runner.run(agent, "Yo! my man, what's up?")
print(res.final_output)

Hey there! Not much, just chilling and ready to help if you got anything. What's up with you, my man?


In [8]:
# Magic of Sessions, this is now the fourth agents sdk primitive
from agents import SQLiteSession

agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
    model = model
)

# Create a session instance with a session ID
session = SQLiteSession("conversation_123")

# First turn
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session
)
print(result.final_output)  # "San Francisco"

# Second turn - agent automatically remembers previous context
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"

# Also works with synchronous runner
result = Runner.run_sync(
    agent,
    "What's the population?",
    session=session
)
print(result.final_output)  # "Approximately 39 million"

San Francisco.
California.
Around 815,000.


In [10]:
result = Runner.run_sync(
    agent,
    "list my previous prompts from this session",
    session=session
)
print(result.final_output)

*   "What city is the Golden Gate Bridge in?"
*   "What state is it in?"
*   "What's the population?"
*   "list my previous prompts"


In [11]:
result = Runner.run_sync(
    agent,
    "which city was it again?",
    session=session
)
print(result.final_output)

San Francisco.


In [12]:
result = Runner.run_sync(
    agent,
    "&?",
    session=session
)
print(result.final_output)

California.


#So now we can see why Sessions is now the 4th primitive of the sdk.
##Now, there are also few methods which we can use to manipulate Sessions

1.   Using pop_item for corrections(edit)
2.   Get all items in a session
3.   Add new items to a session

In [20]:
# Get all items in a session
items = await session.get_items()
print(items)

[{'content': 'What city is the Golden Gate Bridge in?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'San Francisco.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'What state is it in?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'California.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': "What's the population?", 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'Around 815,000.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'list my previous prompts', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "I cannot access your previous prompts. I don't retain information from past interactions.", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': '

In [30]:
# Add new items to a session
new_message = [
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi there!"}
]
await session.add_items(new_message)
print(items)

[{'content': 'What city is the Golden Gate Bridge in?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'San Francisco.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'What state is it in?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'California.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': "What's the population?", 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'Around 815,000.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'list my previous prompts', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "I cannot access your previous prompts. I don't retain information from past interactions.", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': '

In [28]:
# Remove and return the most recent item
last_item = await session.pop_item()
print(last_item)  # {"role": "assistant", "content": "Hi there!"}

{'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'California.', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}


In [29]:
# Clear all items from a session
await session.clear_session()
print(session)